In [ ]:
import os
import pandas as pd
import numpy as np
from uuid import uuid4

import data_generating_process as dgp
import dimensionality_reduction as dr
import calculate_dea as cdea
import evaluate_results as er

In [ ]:
run_serial = str(uuid4())

In [ ]:
parent_dir = '/Users/juanmalagon/repos/high-dim-dea'
results_dir = os.path.join(parent_dir, 'results')

In [ ]:
params_set_wish = {
    'N_list': [20, 50, 100, 500],
    'n_list': [20, 50, 100, 200, 500, 1000],
    'gamma_list': [0.5, 1],
    'rts_list': ['crs', 'vrs'],
}

In [ ]:
params_set_run_1 = {
    'N_list': [100],
    'n_list': [20, 50, 100, 200],
    'gamma_list': [1],
    'rts_list': ['crs'],
    'alpha_1':[0.25]
}

In [ ]:
params_set_run_2 = {
    'N_list': [20],
    'n_list': [20],
    'gamma_list': [1],
    'rts_list': ['crs'],
    'alpha_1':['1/N']
}

In [ ]:
## Set up initial variables

N = 20
M = 1
n = 20
alpha_1 = 1/N
gamma = 1
sigma_u = 0.5
rts='crs'
orientation='input'
nr_simulations = 1_000
seed = 42

In [ ]:
np.random.seed(seed)

In [ ]:
print('INITIAL SETUP \n')
print(f'Number of inputs: {N}')
print(f'Number of outputs: {M}')
print(f'Number of DMUs: {n}')
print(f'Parameter alpha_1: {alpha_1}')
print(f'Parameter gamma: {gamma}')
print(f'Parameter sigma_u: {sigma_u}')
print(f'Return to scale: {rts}')
print(f'Orientation: {orientation}')
print(f'Number of simulations: {nr_simulations}')
print(f'Seed: {seed}')

In [ ]:
params_dict = {
    'N': N,
    'M': M,
    'n': n,
    'alpha_1': alpha_1,
    'gamma': gamma,
    'sigma_u': sigma_u,
    'rts': rts,
    'orientation': orientation,
    'nr_simulations': nr_simulations,
    'seed': seed
}

In [ ]:
def run_simulation(params_dict):

    N = params_dict['N']
    M = params_dict['M']
    n = params_dict['n']
    alpha_1 = params_dict['alpha_1']
    gamma = params_dict['gamma']
    sigma_u = params_dict['sigma_u']
    rts = params_dict['rts']
    orientation = params_dict['orientation']
    seed = params_dict['seed']

    ## Data Generating Process
    data_dict = dgp.generate_data_dict(
        n=n,
        N=N,
        M=M,
        alpha_1=alpha_1,
        gamma=gamma,
        sigma_u=sigma_u,
        verbose=False
    )
    x = data_dict["x"]
    y = data_dict["y"]
    y_tilde = data_dict["y_tilde"]
    efficiency_score_by_design = (y/y_tilde).squeeze()

    ## Dimensionality Reduction
    embeddings = dr.create_embeddings(x=x, seed=seed)
    embeddings_df_dict = embeddings['embeddings_df_dict']
    dims_for_embedding_dict = embeddings['dims_for_embedding_dict']

    ## Calculate DEA
    efficiency_scores_dict = cdea.calculate_dea_for_embeddings(
        embeddings_df_dict=embeddings_df_dict,
        y=y,
        rts=rts,
        orientation=orientation
    )

    ## Evaluate Results
    evaluation_df = er.create_evaluation_df(
        efficiency_scores_dict=efficiency_scores_dict,
        efficiency_score_by_design=efficiency_score_by_design,
        dims_for_embedding_dict=dims_for_embedding_dict,
    )

    return evaluation_df

In [ ]:
evaluation_df_list = []
errors_list = []

for i in range(nr_simulations):
    print(f'Starting iteration {i}')
    try:
        evaluation_df = run_simulation(params_dict)
        evaluation_df['iteration'] = i
        evaluation_df_list.append(evaluation_df)
    except:
        errors_list.append(i)
        print(f'Error in iteration {i}')
    print(f'Finished iteration {i}')

In [ ]:
def export_results(evaluation_df_list, errors_list, params_dict):
    ## Save parameters
    pd.DataFrame(params_dict, index=[0]).to_csv(
        os.path.join(results_dir, f'params_dict_{run_serial}.csv'), index=False)
    
    ## Save errors
    errors_list_df = pd.DataFrame(errors_list, columns=['iteration'])
    errors_list_df.to_csv(
        os.path.join(results_dir, f'errors_list_{run_serial}.csv'), index=False)
    
    ## Save evaluation
    evaluation_df = pd.concat(evaluation_df_list)
    evaluation_df.to_csv(
        os.path.join(results_dir, f'evaluation_df_{run_serial}.csv'),
        index=False)

    ## Save summary
    summary_df = evaluation_df.groupby(['dim_reduction_level', 'dims']).agg(
        {'mae': ['mean', 'std'],
        'spearmanr': ['mean', 'std'],
            'pearsonr': ['mean', 'std'],
            'kendalltau': ['mean', 'std']}).reset_index()
    summary_df.columns = ['dim_reduction_level', 'dims', 'mae_mean', 'mae_std',
                          'spearmanr_mean', 'spearmanr_std', 'pearsonr_mean',
                          'pearsonr_std','kendalltau_mean', 'kendalltau_std']
    summary_df.sort_values(by=['dims', 'dim_reduction_level']).to_csv(
        os.path.join(results_dir, f'summary_df_{run_serial}.csv'), index=False)
    
    return None

In [ ]:
export_results(evaluation_df_list, errors_list, params_dict)